In [ ]:
import base64

import geopandas as gpd
import numpy as np
from folium.plugins import FloatImage
from mapclassify import classify
from matplotlib import pyplot as plt
from matplotlib.colors import rgb2hex
from PIL import Image

from bivario.cmap import (  # cmap_from_bivariate_data,; bilinear_cmap,; bilinear_cmap_from_bivariate_data,; corners_cmap_from_bivariate_data,
    bivariate_from_cmaps,
    bivariate_from_name,
)
from bivario.cmap_old import cmap_from_bivariate_data

In [ ]:
df = gpd.read_parquet("example_data/nyc_bike_trips.parquet")
df

In [ ]:
# xx, yy = np.mgrid[0:5, 0:5]

# # nw.from_native(values_a, series_only=True)
# # xx
# import narwhals as nw

# nw.from_native(xx)

In [ ]:
xx, yy = np.mgrid[0:50, 0:50]
# xx, yy = np.mgrid[0:5, 0:5]
C_map = bivariate_from_cmaps(yy, xx, plt.cm.Oranges, plt.cm.Blues)
# C_map = cmap_from_bivariate_data(yy, xx, plt.cm.Oranges, plt.cm.Blues)
# C_map = bivariate_from_accents(yy, xx)
# C_map = bivariate_from_corners(yy, xx)

fig = plt.figure(figsize=(10, 5))

ax1 = fig.add_subplot(1, 2, 1)
x = xx.flatten()
y = yy.flatten()
cflat = C_map.reshape((len(x), 3))
plt.scatter(x, y, c=cflat, s=8)

ax1.set_title("Data")

# xx, yy = np.mgrid[0:16, 0:16]
# xx, yy = np.mgrid[0:5, 0:5]
xx, yy = np.mgrid[0:3, 0:3]
C_map = bivariate_from_cmaps(xx, yy, plt.cm.Oranges, plt.cm.Blues)
# C_map = bivariate_from_accents(xx, yy, light_mode=True)
# C_map = bivariate_from_corners(xx, yy)

ax2 = fig.add_subplot(1, 2, 2)
ax2.imshow(C_map, origin="lower")
ax2.set_title("Bivariate Color Map")
ax2.set_xlabel("Variable 1")
ax2.set_ylabel("Variable 2")
ax2.set_ylim((-0.5, 0.5 + (yy.max() - yy.min())))

fig.tight_layout()
fig.show()

In [ ]:
binning_start = classify(df["morning_starts"], scheme="NaturalBreaks", k=7)
binning_end = classify(df["morning_ends"], scheme="NaturalBreaks", k=7)
binning_end

In [ ]:
binning_start.bins

In [ ]:
C_map = cmap_from_bivariate_data(
    binning_start.yb, binning_end.yb, cmap1=plt.cm.Oranges, cmap2=plt.cm.Blues
)
C_map

In [ ]:
# cmap1 = plt.cm.YlGn
# # cmap1 = plt.cm.Grays
# # cmap2 = plt.cm.Blues

# # cmap1 = plt.cm.rainbow
# cmap2 = plt.cm.RdPu

# default_grid_size = 5
# # current_grid_size = 128
# current_grid_size = 9
# # current_grid_size = 3
# xx, yy = np.mgrid[0:current_grid_size, 0:current_grid_size]
# # legend_cmap = cmap_from_bivariate_data(xx, yy, cmap1=cmap1, cmap2=cmap2)
# # legend_cmap = bilinear_cmap_from_bivariate_data(xx, yy, c1=[1, 0.5, 0], c2=[0, 0.5, 1])

# c1 = [1, 0.5, 0]
# c2 = [0, 0.5, 1]
# # legend_cmap = bilinear_cmap_from_bivariate_data(xx, yy, c1=c1, c2=c2)
# img = Image.fromarray(np.uint8((legend_cmap)[:, :, :3] * 255))
# display(img.transpose(1).resize(size=(200, 200), resample=0))

# # legend_cmap = bilinear_cmap_from_bivariate_data(xx, yy, c1=c1, c2=c2, dark_end=False)
# img = Image.fromarray(np.uint8((legend_cmap)[:, :, :3] * 255))
# display(img.transpose(1).resize(size=(200, 200), resample=0))

In [ ]:
from bivario.palettes import BIVARIATE_CORNER_PALETTES

# grid_size = 7
# grid_size = 64
grid_size = 7
grid_size = 5
# c1 = [1, 0.5, 0]
# c2 = [0, 0.5, 0.5]
xx, yy = np.mgrid[0:grid_size, 0:grid_size]

# ec = [0.30, 0.40, 0.85]
# c1 = [0.90, 0.40, 0.65]
# c2 = [0.15, 0.75, 0.70]
# sc = [0.95, 0.85, 0.45]

# ec = [0.15, 0.20, 0.50]
# c1 = [0.95, 0.40, 0.20]
# c2 = [0.10, 0.70, 0.65]
# sc = [0.95, 0.85, 0.55]

# c00	Deep indigo	(0.15, 0.20, 0.50)	#263380
# c10	Vivid orange	(0.95, 0.40, 0.20)	#F26633
# c01	Turquoise	(0.10, 0.70, 0.65)	#1AB3A6
# c11	Soft yellow-white	(0.95, 0.85, 0.55)	#F2D98C
total_keys = len(BIVARIATE_CORNER_PALETTES)
cols = 5
fig, axes = plt.subplots(int(np.ceil(total_keys / cols)), cols, figsize=(10, 10))

axes_list = axes.flatten()
for ax in axes_list:
    ax.set_axis_off()

for _i, k in enumerate(BIVARIATE_CORNER_PALETTES.keys()):
    # lcmap = bivariate_from_name(xx, yy, "sunrise_harbor")
    # if k.startswith('s'):
    lcmap = bivariate_from_name(xx, yy, k)
    # lcmap = corners_cmap_from_bivariate_data(
    #     xx,
    #     yy,
    #     # start_colour=[0.30, 0.50, 0.70],
    #     # c1=[0.85, 0.60, 0.15],
    #     # c2=[0.40, 0.70, 0.50],
    #     # end_colour=[0.90, 0.80, 0.45],
    #     # start_colour=[0.25, 0.35, 0.60],
    #     # c1=[0.90, 0.55, 0.00],
    #     # c2=[0.40, 0.70, 0.90],
    #     # end_colour=[0.95, 0.85, 0.55],
    #     start_colour=sc,
    #     c1=c1,
    #     c2=c2,
    #     end_colour=ec,
    # )
    img = Image.fromarray(np.uint8((lcmap)[:, :, :3] * 255))
    axes_list[_i].imshow(img, origin="lower")
    axes_list[_i].set_title(k)
    # axes_list[_i].set_axis_off()
    # print(k)
    # display(img.transpose(1).resize(size=(200, 200), resample=0))

In [ ]:
from itertools import product

from shapely import box

default_grid_size = 9
current_grid_size = 128
# current_grid_size = 7
xx, yy = np.mgrid[0:current_grid_size, 0:current_grid_size]

squares = []
for _i, _j in product(range(default_grid_size), range(default_grid_size)):
    squares.append(dict(geometry=box(_i, _j, _i + 1, _j + 1), x=xx[_j, _i], y=yy[_j, _i]))

squares_gdf = gpd.GeoDataFrame(squares, crs=4326)

legend_cmap = cmap_from_bivariate_data(
    xx, yy, cmap1=plt.cm.Oranges, cmap2=plt.cm.Blues, corner_colour=[255, 0, 0]
)

img = Image.fromarray(np.uint8((legend_cmap)[:, :, :3] * 255))

# img = img.transpose(Image.FLIP_TOP_BOTTOM)  # .resize((100, 100), resample=0)

cmap = cmap_from_bivariate_data(
    squares_gdf["x"],
    squares_gdf["y"],
    # cmap1=plt.cm.YlGn,
    # cmap2=plt.cm.RdPu,
    cmap1=plt.cm.Oranges,
    cmap2=plt.cm.Blues,
    # corner_colour=[255, 0, 0],
)
hex_values = [rgb2hex(cmap[i, :]) for i in range(cmap.shape[0])]

color_dict = dict(zip(squares_gdf.index, hex_values, strict=False))

IMAGE_SIZE_PX = 200
DPI = 100

TICK_FONTSIZE_PX = 10
TICK_FONTSIZE_PT = TICK_FONTSIZE_PX * 72 / DPI

IMG_SIZE_IN = IMAGE_SIZE_PX / DPI


def figure_to_base64_string(plt_fig):
    import io

    buffered = io.BytesIO()
    # img.save(buffered, format="PNG")
    # plt_fig.tight_layout()
    plt_fig.savefig(buffered, transparent=True, dpi=DPI)
    # plt_fig.savefig(buffered, dpi=DPI)
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


color = "white"

with plt.rc_context(
    {
        "axes.labelcolor": color,
        "axes.edgecolor": color,
        "xtick.color": color,
        "ytick.color": color,
    }
):
    fig, ax = plt.subplots(figsize=(IMG_SIZE_IN, IMG_SIZE_IN), dpi=DPI, constrained_layout=True)
    ax.imshow(img, origin="lower")
    ax.tick_params(axis="both", which="both", length=0)
    # ax.axis("off")

    # fig.tight_layout()

    ax.annotate(
        "",
        xy=(0, 1),
        xytext=(0, 0),
        arrowprops=dict(
            arrowstyle="->",
            lw=1,
            color=color,
        ),
        xycoords="axes fraction",
    )
    ax.annotate(
        "",
        xy=(1, 0),
        xytext=(0, 0),
        arrowprops=dict(
            arrowstyle="->",
            lw=1,
            color=color,
        ),
        xycoords="axes fraction",
    )

    ax.set_ylabel("Column B", fontsize=TICK_FONTSIZE_PT)
    ax.set_xlabel("Column A", fontsize=TICK_FONTSIZE_PT)
    ax.tick_params(labelsize=TICK_FONTSIZE_PT)

    # labels = [0, *binning_start.bins]

    # # numerical
    # xticks = np.linspace(-0.5, legend_cmap.shape[0] + 0.5, len(labels))
    # # xticks = np.arange(-0.5, cmap.shape[1], 1)
    # # categorical
    # # xticks = np.arange(0, cmap.shape[1], 1)
    # ax.set_xticks(xticks)

    # ax.set_xticklabels(labels[:len(xticks)], ha="right")
    # ax.set_xticklabels(labels)

    fig.tight_layout()

    fig.canvas.draw()
    renderer = fig.canvas.get_renderer()

    # Get bounding boxes (in display / pixel coordinates)
    bbox_ax = ax.get_window_extent(renderer)  # The axes area (data + ticks + labels)
    bbox_data = ax.get_position()  # Normalized position in figure

    # Compute inner data area size (in pixels)
    data_width_px = bbox_ax.width
    data_height_px = bbox_ax.height

    print(f"Initial data area size: {data_width_px:.1f}×{data_height_px:.1f}px")

    # Calculate scale factor so data area = target_data_px
    scale = IMAGE_SIZE_PX / min(data_width_px, data_height_px)

    print(fig.get_size_inches())

    # Compute new figure size (inches)
    w_in, h_in = fig.get_size_inches()
    new_w_in = w_in * scale
    new_h_in = h_in * scale

    fig.set_size_inches(new_w_in, new_h_in)

    print(fig.get_size_inches())

    # Redraw with new size
    fig.canvas.draw()

    renderer = fig.canvas.get_renderer()

    # Get bounding boxes (in display / pixel coordinates)
    bbox_ax = ax.get_window_extent(renderer)  # The axes area (data + ticks + labels)
    bbox_data = ax.get_position()
    print(bbox_data)  # Normalized position in figure
    print(bbox_ax)
    x0, y0, w, h = bbox_ax.bounds
    print(x0, y0, w, h)  # Normalized position in figure

    # Compute inner data area size (in pixels)
    data_width_px = bbox_ax.width
    data_height_px = bbox_ax.height

    print(f"Changed data area size: {data_width_px:.1f}×{data_height_px:.1f}px")

    data_url = "data:image/png;base64," + figure_to_base64_string(fig)

    plt.close()


# fig.savefig('temp.png', transparent=True)

m = squares_gdf.reset_index().explore(
    # "index", cmap=lambda x: color_dict[x], legend=False, tiles="CartoDB Positron"
    "index",
    cmap=lambda x: color_dict[x],
    legend=False,
    tiles="CartoDB DarkMatter",
    style_kwds=dict(fillOpacity=1.0),
)

# print(data_url)

legend_position_x = 50
legend_position_y = 80

new_fig_size_px = new_w_in * DPI
translateX = legend_position_x - x0
translateY = -(new_fig_size_px - y0 - 200)
translateY = +y0 - legend_position_y
print(translateX, translateY)

FloatImage(
    data_url,
    bottom=0,
    left=0,
    width=f"{new_fig_size_px}px",
    transform=f"translate({translateX}px, {translateY}px)",
).add_to(m)

# plt.close()
# plt.show()
m

In [ ]:
binning_start.bins[-2]

In [ ]:
from folium.plugins import FloatImage
from matplotlib.colors import rgb2hex
from PIL import Image

cmap1 = plt.cm.YlGn
# cmap1 = plt.cm.Oranges
# cmap2 = plt.cm.Blues

# cmap1 = plt.cm.rainbow
cmap2 = plt.cm.Blues
# cmap2 = plt.cm.YlGn

p = "electric_neon"
# p = "coral_ocean"
# p = "suncrest_teal"

binning_start = classify(df["morning_starts"], scheme="NaturalBreaks", k=5)
binning_end = classify(df["morning_ends"], scheme="NaturalBreaks", k=5)

# cmap = cmap_from_bivariate_data(
#     binning_start.yb,
#     binning_end.yb,
#     cmap1=cmap1,
#     cmap2=cmap2,
#     # binning_start.yb, binning_end.yb, cmap1=cmap1, cmap2=cmap2, corner_colour=[3, 44, 252]
# )
# cmap = corners_cmap_from_bivariate_data(
#     binning_start.yb,
#     binning_end.yb,
#     start_colour=sc,
#     c1=c1,
#     c2=c2,
#     end_colour=ec,
# )
cmap = bivariate_from_name(binning_start.yb, binning_end.yb, p)
# cmap = bilinear_cmap_from_bivariate_data(
#     binning_start.yb,
#     binning_end.yb,
#     # start_colour=sc,
#     c1=c1,
#     c2=c2,
#     # end_colour=ec,
# )
hex_values = [rgb2hex(cmap[i, :]) for i in range(cmap.shape[0])]

color_dict = dict(zip(df.index, hex_values, strict=False))
alpha_dict = dict(
    zip(
        df.index,
        np.sqrt(
            np.minimum(
                1,
                np.maximum(
                    # df["morning_starts"] / df["morning_starts"].quantile(0.9),
                    df["morning_starts"] / binning_start.bins[-2],
                    # df["morning_ends"] / df["morning_ends"].quantile(0.9),
                    df["morning_ends"] / binning_end.bins[-2],
                ),
            )
        ),
        strict=False,
    )
)

# default_grid_size = 5
# current_grid_size = 7
current_grid_size = len(binning_start.classes)
# current_grid_size = 5
xx, yy = np.mgrid[0:current_grid_size, 0:current_grid_size]
# legend_cmap = cmap_from_bivariate_data(xx, yy, cmap1=cmap1, cmap2=cmap2, corner_colour=[3, 44, 252])
# legend_cmap = cmap_from_bivariate_data(xx, yy, cmap1=cmap1, cmap2=cmap2)

# legend_cmap = corners_cmap_from_bivariate_data(
#     xx,
#     yy,
#     start_colour=sc,
#     c1=c1,
#     c2=c2,
#     end_colour=ec,
# )
# legend_cmap = bilinear_cmap_from_bivariate_data(
#     xx,
#     yy,
#     # start_colour=sc,
#     c1=c1,
#     c2=c2,
#     # end_colour=ec,
# )
legend_cmap = bivariate_from_name(xx, yy, p)

img = Image.fromarray(np.uint8((legend_cmap)[:, :, :3] * 255))

# img = img.transpose(Image.FLIP_TOP_BOTTOM)  # .resize((100, 100), resample=0)

# params = {"ytick.color" : "w",
#           "xtick.color" : "w",
#           "axes.labelcolor" : "w",
#           "axes.edgecolor" : "w"}

IMAGE_SIZE_PX = 250
DPI = 200

TICK_FONTSIZE_PX = 10
TICK_FONTSIZE_PT = TICK_FONTSIZE_PX * 72 / DPI

IMG_SIZE_IN = IMAGE_SIZE_PX / DPI


def figure_to_base64_string(plt_fig):
    import io

    buffered = io.BytesIO()
    # img.save(buffered, format="PNG")
    # plt_fig.tight_layout()
    plt_fig.savefig(buffered, transparent=True, dpi=DPI)
    # plt_fig.savefig(buffered, dpi=DPI)
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


# color = "white"
color = "black"

with plt.rc_context(
    {
        "axes.labelcolor": color,
        "axes.edgecolor": color,
        "xtick.color": color,
        "ytick.color": color,
    }
):
    fig, ax = plt.subplots(figsize=(IMG_SIZE_IN, IMG_SIZE_IN), dpi=DPI, constrained_layout=True)
    ax.imshow(img, origin="lower")
    ax.tick_params(axis="both", which="both", length=0)
    # ax.axis("off")

    # fig.tight_layout()

    ax.annotate(
        "",
        xy=(0, 1),
        xytext=(0, 0),
        arrowprops=dict(
            arrowstyle="->",
            lw=1,
            color=color,
        ),
        xycoords="axes fraction",
    )
    ax.annotate(
        "",
        xy=(1, 0),
        xytext=(0, 0),
        arrowprops=dict(
            arrowstyle="->",
            lw=1,
            color=color,
        ),
        xycoords="axes fraction",
    )

    ax.set_ylabel("Column B", fontsize=TICK_FONTSIZE_PT)
    ax.set_xlabel("Column A", fontsize=TICK_FONTSIZE_PT)
    ax.tick_params(labelsize=TICK_FONTSIZE_PT)

    labels = [0, *binning_start.bins]

    # numerical
    xticks = np.linspace(-0.5, legend_cmap.shape[1] - 0.5, len(labels))
    # xticks = np.arange(-0.5, cmap.shape[1], 1)
    # categorical
    # xticks = np.arange(0, cmap.shape[1], 1)
    ax.set_xticks(xticks)

    # ax.set_xticklabels(labels[:len(xticks)], ha="right")
    ax.set_xticklabels(labels)

    fig.tight_layout()

    fig.canvas.draw()
    renderer = fig.canvas.get_renderer()

    # Get bounding boxes (in display / pixel coordinates)
    bbox_ax = ax.get_window_extent(renderer)  # The axes area (data + ticks + labels)
    bbox_data = ax.get_position()  # Normalized position in figure

    # Compute inner data area size (in pixels)
    data_width_px = bbox_ax.width
    data_height_px = bbox_ax.height

    print(f"Initial data area size: {data_width_px:.1f}×{data_height_px:.1f}px")

    # Calculate scale factor so data area = target_data_px
    scale = IMAGE_SIZE_PX / min(data_width_px, data_height_px)

    print(fig.get_size_inches())

    # Compute new figure size (inches)
    w_in, h_in = fig.get_size_inches()
    new_w_in = w_in * scale
    new_h_in = h_in * scale

    fig.set_size_inches(new_w_in, new_h_in)

    print(fig.get_size_inches())

    # Redraw with new size
    fig.canvas.draw()

    renderer = fig.canvas.get_renderer()

    # Get bounding boxes (in display / pixel coordinates)
    bbox_ax = ax.get_window_extent(renderer)  # The axes area (data + ticks + labels)
    bbox_data = ax.get_position()
    print(bbox_data)  # Normalized position in figure
    print(bbox_ax)
    x0, y0, w, h = bbox_ax.bounds
    print(x0, y0, w, h)  # Normalized position in figure

    # Compute inner data area size (in pixels)
    data_width_px = bbox_ax.width
    data_height_px = bbox_ax.height

    print(f"Changed data area size: {data_width_px:.1f}×{data_height_px:.1f}px")

    data_url = "data:image/png;base64," + figure_to_base64_string(fig)

    plt.close()


# fig.savefig('temp.png', transparent=True)

m = df.reset_index().explore(
    # "index", cmap=lambda x: color_dict[x], legend=False, tiles="CartoDB Positron"
    "index",
    cmap=lambda x: color_dict[x],
    legend=False,
    # tiles="CartoDB DarkMatter",
    tiles="CartoDB Voyager",
    # alpha=""
    # style_kwds=dict(fillOpacity=0.8),
    style_kwds=dict(
        style_function=lambda x: dict(
            # opacity=alpha_dict[x["properties"]["index"]],
            opacity=0,
            fillOpacity=alpha_dict[x["properties"]["index"]],
        )
    ),
)

# print(data_url)

legend_position_x = 50
legend_position_y = 80

new_fig_size_px = new_w_in * DPI
translateX = legend_position_x - x0
translateY = -(new_fig_size_px - y0 - 200)
translateY = +y0 - legend_position_y
print(translateX, translateY)

FloatImage(
    data_url,
    bottom=0,
    left=0,
    width=f"{new_fig_size_px}px",
    transform=f"translate({translateX}px, {translateY}px)",
    pointer_events="none",
).add_to(m)

# plt.close()
# plt.show()
m

In [ ]:
m.save("bivariate_legend_map.html")

In [ ]:
from matplotlib.colors import rgb2hex

cmap = cmap_from_bivariate_data(
    binning_start.yb,
    binning_end.yb,
    cmap1=plt.cm.Oranges,
    cmap2=plt.cm.Blues,
)
hex_values = [rgb2hex(cmap[i, :]) for i in range(cmap.shape[0])]

color_dict = dict(zip(df.index, hex_values, strict=False))

m = df.reset_index().explore(
    "index", cmap=lambda x: color_dict[x], legend=False, tiles="CartoDB Positron"
)
m